In [1]:
from __future__ import print_function
import os
import sys
import time
import numpy as np
import re
import faiss
from multiprocessing.dummy import Pool as ThreadPool
from matplotlib import pyplot

In [2]:
def mmap_fvecs(fname):
    x = np.memmap(fname, dtype='int32', mode='r')
    d = x[0]
    return x.view('float32').reshape(-1, d + 1)[:, 1:]


def mmap_bvecs(fname):
    x = np.memmap(fname, dtype='uint8', mode='r')
    d = x[:4].view('int32')[0]
    return x.reshape(-1, d + 4)[:, 4:]

def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy()

dbname        = 'SIFT100M'
index_key     = 'OPQ16,IVF1024,PQ16'


tmpdir = '../trained_CPU_indexes/bench_cpu_{}_{}'.format(dbname, index_key)

if not os.path.isdir(tmpdir):
    raise("%s does not exist")


#################################################################
# Prepare dataset
#################################################################


print("Preparing dataset", dbname)

if dbname.startswith('SIFT'):
    # SIFT1M to SIFT1000M
    dbsize = int(dbname[4:-1])
    xb = mmap_bvecs('../bigann/bigann_base.bvecs')
    xq = mmap_bvecs('../bigann/bigann_query.bvecs')
    xt = mmap_bvecs('../bigann/bigann_learn.bvecs')

    # trim xb to correct size
    xb = xb[:dbsize * 1000 * 1000]

    gt = ivecs_read('../bigann/gnd/idx_%dM.ivecs' % dbsize)

elif dbname == 'Deep1B':
    xb = mmap_fvecs('../deep1b/base.fvecs')
    xq = mmap_fvecs('../deep1b/deep1B_queries.fvecs')
    xt = mmap_fvecs('../deep1b/learn.fvecs')
    # deep1B's train is is outrageously big
    xt = xt[:10 * 1000 * 1000]
    gt = ivecs_read('../deep1b/deep1B_groundtruth.ivecs')

else:
    print('unknown dataset', dbname, file=sys.stderr)
    sys.exit(1)


print("sizes: B %s Q %s T %s gt %s" % (
    xb.shape, xq.shape, xt.shape, gt.shape))

nq, d = xq.shape
nb, d = xb.shape
assert gt.shape[0] == nq


#################################################################
# Load Index
#################################################################

def get_populated_index():

    filename = "%s/%s_%s_populated.index" % (
        tmpdir, dbname, index_key)

    if not os.path.exists(filename):
        raise("Index does not exist!")
    else:
        print("loading", filename)
        index = faiss.read_index(filename)
    return index


index = get_populated_index()

Preparing dataset SIFT100M
sizes: B (100000000, 128) Q (10000, 128) T (100000000, 128) gt (10000, 1000)
loading ../trained_CPU_indexes/bench_cpu_SIFT100M_OPQ16,IVF1024,PQ16/SIFT100M_OPQ16,IVF1024,PQ16_populated.index


In [3]:
HBM_bank_num = int(10) 
cluster_num   = 1024
PQ_bytes      = 16

folder_name = 'FPGA_data_{}_{}_HBM_{}_banks'.format(dbname, index_key, HBM_bank_num)


if not os.path.exists('/home/wejiang/saved_npy_data/' + folder_name):
    os.mkdir('/home/wejiang/saved_npy_data/' + folder_name)

In [4]:
""" Part 1: Save index, quantizer, and HBM related contents """

' Part 1: Save index, quantizer, and HBM related contents '

In [5]:
""" Get OPQ Matrix """
linear_trans = faiss.downcast_VectorTransform(index.chain.at(0))
OPQ_mat = faiss.vector_to_array(linear_trans.A)
OPQ_mat = OPQ_mat.reshape((128,128))
OPQ_mat = np.array(OPQ_mat, dtype=np.float32)
print(OPQ_mat, OPQ_mat.shape)

[[ 0.07384154  0.02917186  0.04120482 ...  0.25396776 -0.03011633
  -0.11850409]
 [-0.03223811  0.05099024 -0.02927521 ... -0.03151678  0.03883978
  -0.19506791]
 [ 0.0409503  -0.06328399 -0.04217225 ...  0.045585    0.14044672
  -0.16505861]
 ...
 [ 0.10278489 -0.02464998 -0.01166504 ... -0.02636559 -0.06372313
  -0.00042123]
 [-0.0661429  -0.00035493 -0.02701051 ...  0.00215594 -0.03124848
   0.07937893]
 [-0.18142596  0.04097258 -0.0155439  ...  0.09283708 -0.02791172
  -0.07091404]] (128, 128)


In [6]:
""" Get IVF index (coarse quantizer) and product quantizer"""

downcasted_index = faiss.downcast_index(index.index)

def get_sub_quantizer_centroids(index):
    """
    return the sub-quantizer centroids, 
    shape = (m, 256, d / m)
    e.g., d=128, m=16 -> (16, 256, 8)
    """
    pq = index.pq
    cen = faiss.vector_to_array(pq.centroids)
    cen = cen.reshape(pq.M, pq.ksub, pq.dsub)
    
    return cen

def get_coarse_quantizer_centroids(index):
    """
    return the coarse-grained quantizer centroids,
    shape = (nlist, d),
    e.g., nlist=1024, d=128 -> (1024, 128)
    """
    coarse_quantizer = faiss.downcast_index(index.quantizer)
    coarse_cen = faiss.vector_to_array(coarse_quantizer.xb)

    coarse_cen = coarse_cen.reshape(coarse_quantizer.ntotal, coarse_quantizer.d)
    return coarse_cen

# Get Sub quantizer info
sub_cen = get_sub_quantizer_centroids(downcasted_index)
print("==== Sub-quantizer ====\n{}\n\nshape:{}\n".format(sub_cen, sub_cen.shape))

# Get Coarse quantizer info
coarse_cen = get_coarse_quantizer_centroids(downcasted_index)
print("==== Coarse-quantizer ====\n{}\n\nshape:{}\n".format(coarse_cen, coarse_cen.shape))

==== Sub-quantizer ====
[[[  7.701072   -10.134779    -3.1058064  ... -12.593111   -15.8248
   -13.688884  ]
  [ -9.073574   -26.004753   -14.420386   ...  -5.0264387  -23.60102
     4.7707224 ]
  [ -0.1385611   -2.9603403    2.3556454  ...  -5.370697    -2.195483
     2.4445875 ]
  ...
  [ -1.8908739   30.243452    -2.642206   ...  51.064354    39.028652
    -2.5172977 ]
  [ -7.2881513   23.993711     6.6314507  ...  29.318893    22.761553
    10.165856  ]
  [  7.367509    28.037127     2.3637779  ...  51.40467     15.065649
    29.337631  ]]

 [[-13.7798395   29.15695    -33.73125    ... -38.735394    20.329971
   -54.55682   ]
  [-34.72245      6.9943485  -46.52128    ... -46.70268     42.671432
   -13.341279  ]
  [ 36.810604   -48.830017   -29.457073   ... -36.649597    15.453078
   -48.836193  ]
  ...
  [  3.8649266    1.9249789   12.241467   ...   6.1581097  -10.396231
     5.069171  ]
  [ -0.5296467    0.9181762   -0.6233195  ...   9.847672     5.7698984
    -0.26270878]
  [ 16.

In [7]:
# Save the coarse quantizer and the product quantizer

OPQ_mat = np.array(OPQ_mat, dtype=np.float32)
PQ_quantizer = np.array(sub_cen, dtype=np.float32)
coarse_cen = np.array(coarse_cen, dtype=np.float32)

# 16, 256, 8 -> (0,0,0:8) the first row of the subquantizer of the first sub-vector 
print(OPQ_mat.shape, PQ_quantizer.shape, coarse_cen.shape)

OPQ_mat.tofile("/home/wejiang/saved_npy_data/{}/OPQ_matrix_float32_{}_{}_raw".format(
    folder_name, OPQ_mat.shape[0], OPQ_mat.shape[1]))
PQ_quantizer.tofile("/home/wejiang/saved_npy_data/{}/product_quantizer_float32_{}_{}_{}_raw".format(
    folder_name, PQ_quantizer.shape[0], PQ_quantizer.shape[1], PQ_quantizer.shape[2]))
coarse_cen.tofile("/home/wejiang/saved_npy_data/{}/vector_quantizer_float32_{}_{}_raw".format(
    folder_name, coarse_cen.shape[0], coarse_cen.shape[1]))

(128, 128) (16, 256, 8) (1024, 128)


In [8]:
""" Get contents in a single Voronoi Cell """
invlists = downcasted_index.invlists

def get_invlist(invlists, l):
    """ 
    returns the (vector IDs set, PQ cose set) of list ID "l"
    list_ids: (#vec_in_list, ), e.g., #vec_in_list=10 -> (10, )
    list_codes: (#vec_in_list, m), e.g., #vec_in_list=10, m=16 -> (10, 16)
    
    That the data is *NOT* copied: if the inverted index is deallocated or changes, accessing the array may crash.
    To avoid this, just clone the output arrays on output. 
    """
    ls = invlists.list_size(l)
    list_vec_ids = faiss.rev_swig_ptr(invlists.get_ids(l), ls)
    list_PQ_codes = faiss.rev_swig_ptr(invlists.get_codes(l), ls * invlists.code_size)
    list_PQ_codes = list_PQ_codes.reshape(-1, invlists.code_size)
    return list_vec_ids, list_PQ_codes


In [9]:
# Example of using function "get_invlist"
list_id = 123
list_vec_ids, list_PQ_codes = get_invlist(invlists, list_id)
print("==== Vector IDs ====\n{}\n\nshape: {}\n".format(list_vec_ids, list_vec_ids.shape))
print("==== PQ codes ====\n{}\n\nshape: {}\n".format(list_PQ_codes, list_PQ_codes.shape))

==== Vector IDs ====
[     266     1802     2508 ... 99997527 99997797 99999302]

shape: (82081,)

==== PQ codes ====
[[252 185 212 ... 146 146 136]
 [100  84 206 ... 126 137  96]
 [161 135 103 ... 220 222 212]
 ...
 [137 187  40 ...  28 191 192]
 [ 94  23 148 ... 111 112 237]
 [163 159   7 ... 192  13  93]]

shape: (82081, 16)



In [10]:
def get_contents_to_HBM(invlists, cluster_id, HBM_bank_num=int(21)):
    """
    For a single cluster (list), extract the contents in the format that HBM loads
      inputs:
        invlists: the Faiss index.invlists object
        cluster_id: e.g., 0~8191 for nlist=8192
        HBM_bank_num: 21 for default, athough there are 32 banks on U280, 
                    we don't have enough hardware logic to load and compute at that rate
      outputs:
        HBM_bank_contents( content of 21 banks): a list of 21 element
            each element is a byte object with a set of contents
            the size of the content is m * 64 bytes
            the contents includes (3 * (int32 vector ID) (16 byte PQ code)) + 4byte padding
        entries_per_bank: int, all HBM shares the same number of 512-bit items to scan
        last_valid_element: int from 0 to 62 (63 numbers in total given 21 HBM channels)
            some of the elements in the last row are paddings, which of them is the last non-padding (valid) 
            
      term:
        entry: a 512-bit entry containing 3 PQ codes
        vector: a 20-byte vector containing 4 byte vector ID + 16 byte PQ code
    """
    
    list_vec_ids, list_PQ_codes = get_invlist(invlists, cluster_id)
#     print("list_vec_ids", list_vec_ids.shape)
#     print("list_PQ_codes", list_PQ_codes.shape)
    num_vec = list_vec_ids.shape[0]
    assert list_vec_ids.shape[0] == list_PQ_codes.shape[0]
    
#     print("num_vec", num_vec)
    
    if num_vec % (HBM_bank_num * 3) == 0:
        # no padding
        entries_per_bank = num_vec / (HBM_bank_num * 3)
        last_valid_element = HBM_bank_num * 3 - 1
        num_vec_per_HBM = [int(num_vec / HBM_bank_num)] * HBM_bank_num
        num_pad_per_HBM = [0] * HBM_bank_num
    else:
        # with padding
        entries_per_bank = int(num_vec / (HBM_bank_num * 3)) + 1
        last_valid_element = num_vec % (HBM_bank_num * 3) - 1
        num_vec_per_HBM = []
        num_pad_per_HBM = []
        
        counted_banks = 0
        # bank with full valid elements
        for i in range(int((last_valid_element + 1) / 3)):
            num_vec_per_HBM += [entries_per_bank * 3]
            num_pad_per_HBM += [0]
        counted_banks += int((last_valid_element + 1) / 3)
        
        # (optional) bank with some valid elements and some padding in the last entry
        if (last_valid_element + 1) % 3 != 0:
            num_vec_per_HBM += [(entries_per_bank - 1) * 3 + (last_valid_element + 1) % 3]
            num_pad_per_HBM += [3 - (last_valid_element + 1) % 3]
            counted_banks += 1
        
        # (optional) bank with full padding in the last entry
        for i in range(HBM_bank_num - counted_banks):
            num_vec_per_HBM += [int((entries_per_bank - 1) * 3)]
            num_pad_per_HBM += [3]
            
    assert np.sum(np.array(num_vec_per_HBM)) == num_vec
    assert entries_per_bank * HBM_bank_num * 3 - np.sum(np.array(num_pad_per_HBM)) == num_vec
    
    HBM_bank_contents = []
    
    start = int(0)
    
    zero = int(0)
    empty_byte = zero.to_bytes(1, "little", signed=True)
    
#     print("num_vec_per_HBM:", num_vec_per_HBM)
#     print("num_pad_per_HBM:", num_pad_per_HBM)
    
    for i in range(HBM_bank_num):
        
        # add valid vectors first
        end = start + num_vec_per_HBM[i]
        vec_per_bank_count = 0
        byte_obj = bytes()
        
#         print(start, end)
        
        for vec_id_per_bank in range(start, end):
            
            # Vec ID = signed int
            vec_id = int(list_vec_ids[vec_id_per_bank])
            # Xilinx's ap int use little endian
            # Linux on X86 use little endian
            # https://serverfault.com/questions/163487/how-to-tell-if-a-linux-system-is-big-endian-or-little-endian
            byte_obj += vec_id.to_bytes(4, "little", signed=True)
            
            # PQ code = unsigned char
            PQ_codes = list_PQ_codes[vec_id_per_bank]
            for code in PQ_codes:
                code = int(code)
                # Xilinx's ap int use little endian
                byte_obj += code.to_bytes(1, "little", signed=False)
            
            vec_per_bank_count += 1
            if vec_per_bank_count % 3 == 0:
                byte_obj += empty_byte * 4
        
        start = end
        
        # then add paddings
        if num_pad_per_HBM[i] > 0:
            for pad_id in range(num_pad_per_HBM[i]):
                byte_obj += empty_byte * 20
            byte_obj += empty_byte * 4
        
        HBM_bank_contents += [byte_obj]
       
    for i in range(HBM_bank_num):
        assert len(HBM_bank_contents[i]) == len(HBM_bank_contents[0])
        assert len(HBM_bank_contents[i]) == 64 * entries_per_bank
    
    return HBM_bank_contents, entries_per_bank, last_valid_element

In [11]:
# Get HBM contents from all clusters

list_HBM_bank_contents = [] # array of cluster_num * HBM_bank_num elements
list_entries_per_bank = []
list_last_valid_element = []

for c in range(cluster_num):
    print(c)
    HBM_bank_contents, entries_per_bank, last_valid_element = get_contents_to_HBM(invlists, c, HBM_bank_num)
    list_HBM_bank_contents += HBM_bank_contents
    list_entries_per_bank += [entries_per_bank]
    list_last_valid_element += [last_valid_element]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [12]:
# Reorder list_HBM_bank_contents

print(len(list_HBM_bank_contents))
print("list_entries_per_bank:\n", list_entries_per_bank)
print("list_last_valid_element:\n", list_last_valid_element)

list_HBM_bank_contents_reordered = [] # put all contents of the same HBM bank together

for b in range(HBM_bank_num):
    sub_list = []
    for c in range(cluster_num):
        sub_list += [list_HBM_bank_contents[c * HBM_bank_num + b]]
    print(len(sub_list), len(sub_list[0]))
    list_HBM_bank_contents_reordered += [sub_list]
    
print("list_HBM_bank_contents_reordered:", len(list_HBM_bank_contents_reordered), len(list_HBM_bank_contents_reordered[0]))

10240
list_entries_per_bank:
 [2657, 2398, 2905, 3150, 3440, 3407, 3792, 3904, 2770, 3943, 3378, 5798, 3561, 3118, 3319.0, 3093, 2392, 6488, 3710, 3690, 2573, 9037, 2498, 2883, 3882, 4586, 2806, 2531, 2842, 2799, 1617, 3436, 2382, 3314, 3721, 2510, 4831, 2762, 2665, 2680, 3360, 3160, 3053, 2824, 1865, 3106, 3210, 2989, 2850, 3152, 2489, 2531, 2873, 2826, 2655, 3991, 3841.0, 3465, 3448, 2456, 3348, 2981, 2842, 4898, 645.0, 4199, 2947, 2290, 2828, 2431, 3783, 2577, 2714, 2630, 2609, 3844, 3299, 3163, 4686, 3869, 3941, 3021, 3447, 2773, 9703, 2421, 3692, 9071, 4345, 3487.0, 3469, 6002, 2925, 3346, 9379, 3363, 2622, 2892, 2627, 5712, 2581, 3144, 3409, 3352, 3305, 3281, 3874, 3017.0, 3228, 2726, 2427, 3003, 3794, 2518, 2756, 3632, 6204, 2624, 4456, 2678, 2388, 2367, 3083, 2737, 3416, 3798, 3420, 2964, 2544, 3317, 4790, 3797, 2789, 3013, 3152, 2798, 2661, 1613, 3355, 2767, 3281, 2470.0, 3171, 3448, 2423, 3505, 4008, 3364, 3002, 2841, 3034, 2968, 2090, 2778.0, 4460, 2920, 3455, 2920, 3499, 43

In [13]:
# Concatenate 

HBM_bank_contents_all = [bytes()] * HBM_bank_num # contents of each bank
for b in range(HBM_bank_num):
    HBM_bank_contents_all[b] = HBM_bank_contents_all[b].join(list_HBM_bank_contents_reordered[b])
    
total_size = np.sum(np.array([len(h) for h in HBM_bank_contents_all]))
print("HBM_bank_contents_all: shape: {}\tsize: {}".format(len(HBM_bank_contents_all), total_size))

HBM_bank_contents_all: shape: 10	size: 2133653120


In [14]:
# Save HBM contents 

for b in range(HBM_bank_num):
    assert len(HBM_bank_contents_all[b]) == len(HBM_bank_contents_all[0])

for b in range(HBM_bank_num):
    with open ('/home/wejiang/saved_npy_data/{}/HBM_bank_{}_raw'.format(folder_name, b), 'wb') as f:
        f.write(HBM_bank_contents_all[b])

In [15]:
# Save control contents

#  The format of storing HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid: 
#     8192 start_addr, then 8192 scanned_entries_every_cell, then 8192 last_valid_element
#     int start_addr_LUT[nlist];
#     int scanned_entries_every_cell_LUT[nlist];
#     int last_valid_channel_LUT[nlist];  

list_start_addr_every_cell = [0]
for c in range(cluster_num - 1):
    list_start_addr_every_cell.append(list_start_addr_every_cell[c] + list_entries_per_bank[c])

assert len(list_start_addr_every_cell) == len(list_entries_per_bank) and\
    len(list_start_addr_every_cell) == len(list_last_valid_element)

print(list_start_addr_every_cell[-1])

HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid = \
    list_start_addr_every_cell + list_entries_per_bank + list_last_valid_element

HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid = np.array(
    HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid, dtype=np.int32)

HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid.tofile(
    "/home/wejiang/saved_npy_data/{}/HBM_info_start_addr_and_scanned_entries_every_cell_and_last_element_valid_3_by_{}_raw".format(
        folder_name, cluster_num))

3330992.0


In [16]:
""" Part 2: Save results of different nprobe """

' Part 2: Save results of different nprobe '

In [17]:
#################################################################
# Perform searches and save results
#################################################################


parametersets = ['nprobe=13', 'nprobe=17', 'nprobe=32']

topK=10

ps = faiss.ParameterSpace()
ps.initialize(index)

# make sure queries are in RAM
xq = xq.astype('float32').copy()
xq = np.array(xq, dtype=np.float32)
xq.tofile("/home/wejiang/saved_npy_data/{}/query_vectors_float32_{}_{}_raw".format(folder_name, xq.shape[0], xq.shape[1]))

# a static C++ object that collects statistics about searches
ivfpq_stats = faiss.cvar.indexIVFPQ_stats
ivf_stats = faiss.cvar.indexIVF_stats


# we do queries in a single thread
faiss.omp_set_num_threads(1)

print(' ' * len(parametersets[0]), '\t', 'R@10   time    %pass')
# print(' ' * len(parametersets[0]), '\t', 'R@1    R@10   R@100     time    %pass')

for param in parametersets:
    print(param, '\t', end=' ')
    sys.stdout.flush()
    ps.set_index_parameters(index, param)
    t0 = time.time()
    ivfpq_stats.reset()
    ivf_stats.reset()
    faiss_result_dist, faiss_result_idx = index.search(xq, topK)
    t1 = time.time()
    for rank in [topK]:
        n_ok = (faiss_result_idx[:, :rank] == gt[:, :1]).sum()
        print("%.4f" % (n_ok / float(nq)), end=' ')
    print("%8.3f  " % ((t1 - t0) * 1000.0 / nq), end=' ')
    print("%5.2f" % (ivfpq_stats.n_hamming_pass * 100.0 / ivf_stats.ndis))
    
    D = np.array(faiss_result_dist, dtype=np.float32)
    I = np.array(faiss_result_idx, dtype=np.int32)
    print(D.shape, I.shape, xq.shape)
    D.tofile("/home/wejiang/saved_npy_data/{}/result_nprobe_{}_distance_float32_{}_{}_raw".format(
        folder_name, param[-2:], D.shape[0], D.shape[1]))
    I.tofile("/home/wejiang/saved_npy_data/{}/result_nprobe_{}_index_int32_{}_{}_raw".format(
        folder_name, param[-2:], I.shape[0], I.shape[1]))

          	 R@10   time    %pass
nprobe=13 	 0.8023   16.789    0.00
(10000, 10) (10000, 10) (10000, 128)
nprobe=17 	 0.8076   21.144    0.00
(10000, 10) (10000, 10) (10000, 128)
nprobe=32 	 0.8147   38.982    0.00
(10000, 10) (10000, 10) (10000, 128)
